In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import gc
# 计算分类正确率
from sklearn.linear_model import SGDClassifier
from MeanEncoder import MeanEncoder
from sklearn.preprocessing import LabelEncoder
from dummyPy import OneHotEncoder
import pickle
from sklearn.ensemble import GradientBoostingClassifier

In [19]:
c_test  = "./test_after_features.csv"
## subsample training set
c_sub_train = "./train_sam_after_features.csv"

label_encode = './label_encode'
# label_encode_all = './label_encode_all'
oh_enc_gbdt = './one_hot_1'

## pre-trained model storing
lr_model_save = "./lr_model_1"
gbdt_model_save = "./gbdt_model_1"

## submission files
sub_gbdt = "./GBDT_submission.csv"
sub_gbdt_lr = "./GBDT-LR_submission.csv"

In [3]:
%%time
df_train = pd.read_csv(c_sub_train)#读入已经采样和特征增加的训练数据

Wall time: 39.5 s


In [17]:
df_train.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C18,C19,C20,C21,size,hour1,weekday,time_period,app_site_id,app_site_id_model
0,1.486401e+19,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,18cf06a0,d9b5648e,...,0,551,100217,46,16000,0,1,0,18cf06a0_85f751fd,18cf06a0_85f751fd_f51246a7
1,3.302305e+18,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,...,2,39,100049,32,16000,0,1,0,e2fcccd2_85f751fd,e2fcccd2_85f751fd_be6db1d7
2,1.103181e+19,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,51cedd4e,aefc06bd,...,3,427,100000,61,16000,0,1,0,51cedd4e_85f751fd,51cedd4e_85f751fd_542422a7
3,3.407025e+18,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,...,2,35,100131,32,16000,0,1,0,e2fcccd2_85f751fd,e2fcccd2_85f751fd_d3b3178d
4,1.094784e+18,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,0acbeaa3,45a51db4,...,0,35,100034,157,16000,0,1,0,0acbeaa3_85f751fd,0acbeaa3_85f751fd_0dedddc5


In [20]:
y_all = df_train["click"]
x_train = df_train.drop(["id","click","hour","C15","C16"],axis=1)#去掉一些特征
columns = x_train.columns

In [21]:
columns

Index(['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id',
       'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model',
       'device_type', 'device_conn_type', 'C14', 'C17', 'C18', 'C19', 'C20',
       'C21', 'size', 'hour1', 'weekday', 'time_period', 'app_site_id',
       'app_site_id_model'],
      dtype='object')

In [22]:
columns_ob = ["site_id","site_domain",'C14', 'C17','C21','C19','C20',
                    "site_category","app_id",
                    "app_domain","app_category","device_id","device_ip","device_model","app_site_id","app_site_id_model"]

In [23]:
me = MeanEncoder(columns_ob,target_type='classification')#对于一些高基数类别特征的预处理

In [24]:
%%time
x_train_me = me.fit_transform(x_train,y_all).drop(columns_ob,axis=1)

C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:39: RuntimeWarning: overflow encountered in exp
  self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:39: RuntimeWarning: overflow encountered in exp
  self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
C:\Users\seanz\Documents\WORKFILE\CUHK

C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:39: RuntimeWarning: overflow encountered in exp
  self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:39: RuntimeWarning: overflow encountered in exp
  self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
C:\Users\seanz\Documents\WORKFILE\CUHKSZ\practice\avazu_ctr\MeanEncoder.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
C:\Users\seanz\Documents\WORKFILE\CUHK

Wall time: 8min 48s


In [14]:
x_train_me.head()

,C1,banner_pos,device_type,device_conn_type,C18,C19,C20,C21,size,hour1,...,device_id_pred_0,device_id_pred_1,device_ip_pred_0,device_ip_pred_1,device_model_pred_0,device_model_pred_1,app_site_id_pred_0,app_site_id_pred_1,app_site_id_model_pred_0,app_site_id_model_pred_1
0,1005,0,1,0,0,551,100217,46,16000,0,...,0.824843,0.175157,0.915055,0.084945,0.897594,0.102406,0.923075,0.076925,0.915055,8.494467e-02
1,1005,0,1,0,2,39,100049,32,16000,0,...,0.830111,0.169889,0.830111,0.169889,0.811199,0.188801,0.800059,0.199941,0.777518,2.224824e-01
2,1005,0,1,0,3,427,100000,61,16000,0,...,0.824843,0.175157,0.830111,0.169889,0.936697,0.063303,0.948653,0.051347,0.945119,5.488069e-02
3,1005,0,1,2,2,35,100131,32,16000,0,...,0.824843,0.175157,0.606860,0.393140,0.858411,0.141589,0.800059,0.199941,1.000000,1.911853e-08
4,1005,0,1,2,0,35,100034,157,16000,0,...,0.824843,0.175157,0.830111,0.169889,0.863985,0.136015,0.762568,0.237432,0.857143,1.428571e-01


In [25]:
del x_train
gc.collect()

5886

In [26]:
cols = ['C1', 
        'banner_pos', 
        'device_type', 
        'device_conn_type',
       'C18']

In [15]:
%%time
le = {}
## label transform for training set
for col in cols:
    try:
        ##在对一部分特征进行labelencode处理
        le[col] = LabelEncoder()
        le[col].fit(x_train_me[col])
        x_train_me[col] = le[col].transform(x_train_me[col].values)
    except Exception as e:
        print(col)
        print(e)
pickle.dump(le, open(label_encode, 'wb'))

Wall time: 2.33 s


In [23]:
# %%time
# la_en = pickle.load(open(label_encode, 'rb'))
# ## label transform for training set
# for col in cols:
#     try:
#         ##在对一部分特征进行labelencode处理
#         x_train_me[col] = la_en[col].transform(x_train_me[col].values)
#     except Exception as e:
#         print(col)
#         print(e)

Wall time: 2.34 s


In [23]:
x_train_me.head()

,C1,banner_pos,device_type,device_conn_type,C18,size,hour1,weekday,time_period,site_id_pred_0,...,device_id_pred_0,device_id_pred_1,device_ip_pred_0,device_ip_pred_1,device_model_pred_0,device_model_pred_1,app_site_id_pred_0,app_site_id_pred_1,app_site_id_model_pred_0,app_site_id_model_pred_1
0,2,0,1,0,0,16000,0,1,0,0.88493,...,0.824843,0.175157,0.915055,0.084945,0.897594,0.102406,0.923075,0.076925,0.915055,8.494467e-02
1,2,0,1,0,2,16000,0,1,0,0.88493,...,0.830111,0.169889,0.830111,0.169889,0.811199,0.188801,0.800059,0.199941,0.777518,2.224824e-01
2,2,0,1,0,3,16000,0,1,0,0.88493,...,0.824843,0.175157,0.830111,0.169889,0.936697,0.063303,0.948653,0.051347,0.945119,5.488069e-02
3,2,0,1,1,2,16000,0,1,0,0.88493,...,0.824843,0.175157,0.606860,0.393140,0.858411,0.141589,0.800059,0.199941,1.000000,1.911853e-08
4,2,0,1,1,0,16000,0,1,0,0.88493,...,0.824843,0.175157,0.830111,0.169889,0.863985,0.136015,0.762568,0.237432,0.857143,1.428571e-01


In [50]:
save_after_encode = pd.concat([y_all,x_train_me.head()],axis = 1)
save_after_encode.to_csv('./train_after_encode.csv')

In [51]:
save_after_encode.head()

,click,C1,banner_pos,device_type,device_conn_type,C18,size,hour1,weekday,time_period,...,device_id_pred_0,device_id_pred_1,device_ip_pred_0,device_ip_pred_1,device_model_pred_0,device_model_pred_1,app_site_id_pred_0,app_site_id_pred_1,app_site_id_model_pred_0,app_site_id_model_pred_1
0,0,2.0,0.0,1.0,0.0,0.0,16000.0,0.0,1.0,0.0,...,0.824843,0.175157,0.915055,0.084945,0.897594,0.102406,0.923075,0.076925,0.915055,8.494467e-02
1,0,2.0,0.0,1.0,0.0,2.0,16000.0,0.0,1.0,0.0,...,0.830111,0.169889,0.830111,0.169889,0.811199,0.188801,0.800059,0.199941,0.777518,2.224824e-01
2,0,2.0,0.0,1.0,0.0,3.0,16000.0,0.0,1.0,0.0,...,0.824843,0.175157,0.830111,0.169889,0.936697,0.063303,0.948653,0.051347,0.945119,5.488069e-02
3,0,2.0,0.0,1.0,1.0,2.0,16000.0,0.0,1.0,0.0,...,0.824843,0.175157,0.606860,0.393140,0.858411,0.141589,0.800059,0.199941,1.000000,1.911853e-08
4,0,2.0,0.0,1.0,1.0,0.0,16000.0,0.0,1.0,0.0,...,0.824843,0.175157,0.830111,0.169889,0.863985,0.136015,0.762568,0.237432,0.857143,1.428571e-01


In [21]:
## split the data for the training of GBDT and LR respectively 
## to avoid over-fitting
## 划分数据集
X_train_org = x_train_me.get_values()
y_train_org = y_all.get_values()
X_train, X_valid, y_train, y_valid = train_test_split(X_train_org, y_train_org, test_size = 0.3, random_state = 0)
X_train_gbdt, X_train_lr, y_train_gbdt, y_train_lr = train_test_split(X_train, y_train, test_size = 0.5, random_state = 0)


In [22]:
del x_train_me
del y_all
del X_train
del y_train
gc.collect()
#只需要用来训练gbdt和lr的数据

99

In [23]:
param = {  # init the hyperparams of GBDT
    'learning_rate': 0.1,
    'n_estimators': 60,  # number of trees here
    'max_depth': 8,  # set max_depth of a tree
    'min_samples_split': 500, 
    'min_samples_leaf': 50,
    'subsample': 0.8, 
    'max_leaf_nodes': None,  # set max leaf nodes of a tree
    'random_state': 1,
    'verbose': 0
    }

gbdt_model = GradientBoostingClassifier()
gbdt_model.set_params(**param)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=50, min_samples_split=500,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=1,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [24]:
%%time
## fitting
gbdt_model.fit(X_train_gbdt, y_train_gbdt)

## log-loss of training
y_pred_gbdt = gbdt_model.predict_proba(X_train_gbdt)[:, 1]
log_loss_gbdt = log_loss(y_train_gbdt, y_pred_gbdt)
print('log loss of GBDT on train set: %.5f' % log_loss_gbdt)

y_pred_gbdt = gbdt_model.predict_proba(X_valid)[:, 1]
log_loss_gbdt = log_loss(y_valid, y_pred_gbdt)
print('log loss of GBDT on valid set: %.5f' % log_loss_gbdt)

log loss of GBDT on train set: 0.37713
log loss of GBDT on valid set: 0.38304
Wall time: 1h 22min 32s


In [25]:
## store the pre-trained gbdt_model
pickle.dump(gbdt_model, open(gbdt_model_save, 'wb'))

del X_train_gbdt
del y_train_gbdt
gc.collect()

0

In [26]:
gbdt_model = pickle.load(open(gbdt_model_save, 'rb'))
#----- data for LR (one-hot encoding with GDBT output) -----#
id_cols = []
for i in range(1, gbdt_model.get_params()['n_estimators']+1):
    id_cols.append('tree'+str(i))
oh_enc = OneHotEncoder(id_cols)
def chunker(seq, size):
    return (seq[pos: pos + size] for pos in range(0, len(seq), size))

## oh_enc fit the train_set
df_train_id = pd.DataFrame(gbdt_model.apply(X_train_org)[:, :, 0], columns=id_cols, dtype=np.int8)

for chunk in chunker(df_train_id, 50000):
    oh_enc.fit(chunk)

In [27]:
pickle.dump(oh_enc, open(oh_enc_gbdt, 'wb'))

In [28]:
del df_train_id
del X_train_org
del y_train_org
gc.collect()

0

In [60]:
%%time
#---- LR model -----#
lr_model = SGDClassifier(loss='log')  # using log-loss for LogisticRegression

## input data (one-hot encoding)
df_X_train_lr_id = pd.DataFrame(gbdt_model.apply(X_train_lr)[:, :, 0], columns=id_cols, dtype=np.int8)
df_X_train_lr_id['click'] = y_train_lr

## fitting
for chunk in chunker(df_X_train_lr_id,10000):
    X_train = oh_enc.transform(chunk[id_cols])
    y_train = chunk['click'].astype('int')
    lr_model.partial_fit(X_train, y_train, classes = [0,1])

Wall time: 36min 27s


In [61]:
## store the pre-trained model
pickle.dump(lr_model, open(lr_model_save, 'wb'))

In [62]:
del df_X_train_lr_id
gc.collect()

14

In [27]:
gbdt_model = pickle.load(open(gbdt_model_save, 'rb'))
# lr_model = pickle.load(open(lr_model_save, 'rb'))

In [28]:
id_cols = []
for i in range(1, gbdt_model.get_params()['n_estimators']+1):
    id_cols.append('tree'+str(i))
oh_enc = pickle.load(open(oh_enc_gbdt, 'rb'))

In [29]:
la_en = pickle.load(open(label_encode, 'rb'))

In [ ]:
## oh_enc fit the test_set
df_test_f = pd.read_csv(c_test, 
                        index_col=None,  dtype={'id':str},
                        chunksize=50000, iterator=True)

for chunk in df_test_f:
    ## label transform for training set
    chunk = chunk.drop(["hour","C15","C16"],axis=1)#去掉一些特征
    chunk = me.transform(chunk).drop(columns_ob,axis=1)
    print(chunk.head())
    for col in cols:
        try:
            print(col)
            chunk[col] = la_en[col].transform(chunk[col].values)
        except Exception as e:
            print(e)
            
    X_test = chunk.drop(['id'],axis=1).get_values()
    
#     #----- GBDT-LR -----#
    
#     df_X_test_id = pd.DataFrame(gbdt_model.apply(X_test)[:, :, 0], columns=id_cols, dtype=np.int8)  # gbdt
#     oh_enc.fit(df_X_test_id)
    break

# ## store the encoder
# pickle.dump(oh_enc, open(oh_enc_gbdt, 'wb'))

# oh_enc = pickle.load(open(oh_enc_gbdt, 'rb'))

In [30]:
%%time
##==================== Prediction ====================##
df_test = pd.read_csv(c_test, 
                        index_col=None,dtype = {'id':str},chunksize=1000, iterator=True)        

hd = True
for chunk in df_test:
    ## label transform for training set
#     chunk['id'] = chunk['id'].apply(lambda x: '{:.0f}'.format(x))
    chunk = chunk.drop(["hour","C15","C16"],axis=1)#去掉一些特征
    chunk = me.transform(chunk).drop(columns_ob,axis=1)
    for col in cols:
        chunk[col] = la_en[col].transform(chunk[col].values)
        
    X_test = chunk.drop(['id'],axis=1).get_values()

    #----- GBDT-LR -----#
    y_pred_gbdt = gbdt_model.predict_proba(X_test)[:, 1]
#     X_test_gbdt = pd.DataFrame(gbdt_model.apply(X_test)[:, :, 0], columns=id_cols, dtype=np.int8)
#     print(X_test_gbdt)
#     X2_test = oh_enc.transform(X_test_gbdt)  # one-hot
#     y_pred_lr = lr_model.predict_proba(X2_test)[:, 1]  # lr   
#     print(y_pred_gbdt)
    #----- generation of submission -----#
    chunk['click'] = y_pred_gbdt
#     chunk['click_lr'] = y_pred_lr

    with open(sub_gbdt, 'a') as f: 
        chunk.to_csv(f, columns=['id', 'click'], header=hd, index=False)
#     with open(sub_gbdt_lr, 'a') as f: 
#         chunk.to_csv(f, columns=['id', 'click_lr'], header=hd, index=False)
    hd = False
#     print(chunk)
#     break

Wall time: 12h 2min 1s


In [31]:
sum(1 for line in open('./GBDT_submission.csv')) - 1 

4577464

In [33]:
sum(1 for line in open('./GBDT-LR_submission.csv')) - 1 

4577464

In [32]:
sum(1 for line in open('./test.csv')) - 1 

4577464